## Оптимизация выполнения кода, векторизация, Numba

## Задачи для совместного разбора

In [ ]:
import line_profiler
import numpy as np
from numba import jit
import pandas as pd
import random

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
import numpy as np

N = 1000000  # количество элементов в массиве
A = np.random.randint(low=0, high=1001, size=N)  # генерируем массив A
B = A + 100  # создаем массив B путем прибавления 100 к каждому элементу A
mean_B = np.mean(B)  # вычисляем среднее значение B

print(f"Среднее значение массива B: {mean_B}")

Среднее значение массива B: 600.07043


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
import pandas as pd
import random
import string

df = pd.DataFrame({'col1': [random.random() for _ in range(2000000)],
                   'col2': [random.random() for _ in range(2000000)],
                   'col3': [random.random() for _ in range(2000000)],
                   'col4': [random.random() for _ in range(2000000)]})


def random_string(length):
    return ''.join(random.choice(string.ascii_lowercase) for _ in range(length))

df['key'] = [random_string(5) for _ in range(2000000)]

subset = df[df['key'].str.match('[a-e]{5}')]
subset

,col1,col2,col3,col4,key
220,0.746907,0.137454,0.694848,0.617933,ccdce
740,0.619707,0.746347,0.725800,0.808303,bcdca
4207,0.388938,0.757840,0.071220,0.322288,daeae
7069,0.200107,0.040534,0.101539,0.933318,ccbde
17236,0.163946,0.305243,0.419774,0.261557,bdcac
...,...,...,...,...,...
1982999,0.988387,0.993828,0.744437,0.536347,ccdae
1987447,0.339375,0.482659,0.005229,0.467711,abcce
1992513,0.462057,0.004250,0.563716,0.849436,bdbca
1994589,0.023088,0.573124,0.219409,0.893734,dcddd


## Лабораторная работа 3

In [ ]:
!pip install line_profiler
%load_ext line_profiler


[notice] A new release of pip available: 22.3 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [ ]:
import pandas as pd
import timeit
import time

In [ ]:
# Загрузка таблицы с рецептами
recipes = pd.read_csv('recipes_sample.csv')

# Загрузка таблицы с отзывами
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

recipes['submitted'] = pd.to_datetime(recipes['submitted'])

reviews_2010 = reviews.loc[pd.DatetimeIndex(reviews['date']).year == 2010]

def mean_rating_A(reviews):
    total = 0
    for _, row in reviews.iterrows():
        total += row['rating']
    return total / len(reviews)

def mean_rating_B(reviews_2010):
    total = 0
    for _, row in reviews_2010.iterrows():
        total += row['rating']
    return total / len(reviews_2010)

def mean_rating_C(reviews_2010):
    return reviews_2010['rating'].mean()


# Замер времени выполнения функции mean_rating_A
time_A = timeit.timeit(lambda: mean_rating_A(reviews_2010), number=1)

# Замер времени выполнения функции mean_rating_B
time_B = timeit.timeit(lambda: mean_rating_B(reviews_2010), number=1)

# Замер времени выполнения функции mean_rating_C
time_C = timeit.timeit(lambda: mean_rating_C(reviews_2010), number=1)

# Вывод результатов замеров времени
print('mean_rating_A time:', time_A)
print('mean_rating_B time:', time_B)
print('mean_rating_C time:', time_C)

mean_rating_A time: 1.5684648999999808
mean_rating_B time: 1.5728296000000341
mean_rating_C time: 0.05699000000004162


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [ ]:
%lprun -f mean_rating_A mean_rating_A(reviews_2010)

![Screenshot_1.png](attachment:Screenshot_1.png)

In [ ]:
%lprun -f mean_rating_B mean_rating_B(reviews_2010)

![Screenshot_2.png](attachment:Screenshot_2.png)

In [ ]:
%lprun -f mean_rating_C mean_rating_C(reviews_2010)

![Screenshot_3.png](attachment:Screenshot_3.png)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

print(timeit.timeit(lambda: get_word_reviews_count(reviews_2010), number=1))

5.088115300000027


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

df4 = reviews[['recipe_id', 'rating']].dropna()
df4 = df4[df4['rating'] != 0]
df4 = pd.merge(df4, df4.groupby(['recipe_id'])['rating'].mean().rename('mean_rating'), how='left', left_on='recipe_id', right_index=True)
df4

,recipe_id,rating,mean_rating
370476,57993,5,4.818182
624300,142201,5,5.000000
187037,252013,4,4.000000
706134,404716,5,4.555556
312179,129396,5,5.000000
...,...,...,...
344676,314698,5,5.000000
1013457,335534,4,3.833333
1059834,222001,5,5.000000
453285,354979,5,4.064516


In [ ]:
df4_a = df4['rating'].to_numpy()
df4_b = df4['mean_rating'].to_numpy()

In [ ]:
def mape_a():
    n = len(df4_a)
    return sum([abs(df4_a[i] - df4_b[i]) / df4_a[i] for i in range(n)]) / n * 100
mape_a()

11.17155025905884

In [ ]:
%timeit mape_a()

1.96 s ± 5.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
@jit(nopython = True)
def mape_b():
    n = len(df4_a)
    return sum([abs(df4_a[i] - df4_b[i]) / df4_a[i] for i in range(n)]) / n * 100
mape_b()

11.17155025905884

In [ ]:
%timeit mape_b()

In [ ]:
def mape_c():
    return np.mean(np.fabs(df4_a - df4_b) / df4_a) * 100
mape_c()

In [ ]:
%timeit mape_c()

In [ ]:
@jit(nopython = True)
def mape_d():
    return np.mean(np.fabs(df4_a - df4_b) / df4_a) * 100
mape_d()

In [ ]:
%timeit mape_d()